In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from transformers import pipeline

# Load your CSV file
file_path = 'low.csv'
data = pd.read_csv(file_path)
model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")


/home/sagemaker-user/.conda/envs/pump/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


# Each query must come with a one-sentence instruction that describes the task
task = 'Given a persona query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'How to bake a chocolate cake'),
    get_detailed_instruct(task, 'Symptoms of the flu')
]
# No need to add instruction for retrieval documents
passages = [
    "To bake a delicious chocolate cake, you'll need the following ingredients: all-purpose flour, sugar, cocoa powder, baking powder, baking soda, salt, eggs, milk, vegetable oil, and vanilla extract. Start by preheating your oven to 350°F (175°C). In a mixing bowl, combine the dry ingredients (flour, sugar, cocoa powder, baking powder, baking soda, and salt). In a separate bowl, whisk together the wet ingredients (eggs, milk, vegetable oil, and vanilla extract). Gradually add the wet mixture to the dry ingredients, stirring until well combined. Pour the batter into a greased cake pan and bake for 30-35 minutes. Let it cool before frosting with your favorite chocolate frosting. Enjoy your homemade chocolate cake!",
    "The flu, or influenza, is an illness caused by influenza viruses. Common symptoms of the flu include a high fever, chills, cough, sore throat, runny or stuffy nose, body aches, headache, fatigue, and sometimes nausea and vomiting. These symptoms can come on suddenly and are usually more severe than the common cold. It's important to get plenty of rest, stay hydrated, and consult a healthcare professional if you suspect you have the flu. In some cases, antiviral medications can help alleviate symptoms and reduce the duration of the illness."
]

embeddings = model.encode(queries + passages)
scores = model.similarity(embeddings[:2], embeddings[2:]) * 100
print(scores.tolist())
# [[40.13203811645508, 25.032546997070312], [15.00684642791748, 39.937339782714844]]


In [ ]:

# Preprocess text
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(data['description'].tolist())

# Clustering
num_clusters = 10  # Set the appropriate number of clusters
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(embeddings)
data['cluster'] = clustering_model.labels_

# Summarization
summarizer = pipeline('summarization')
def summarize_text(texts):
    combined_text = " ".join(texts)
    summary = summarizer(combined_text, max_length=50, min_length=25, do_sample=False)
    return summary[0]['summary_text']

data['summary'] = data.groupby('cluster')['candidate_values'].transform(lambda x: summarize_text(x.tolist()))

# Save the result
data.to_csv('/mnt/data/merged_user_persona_dimensions.csv', index=False)
